<a href="https://colab.research.google.com/github/aderald0/Dio_BootcampBradesco/blob/main/Assistente_de_Voz_Multi_Idiomas_Com_Whisper_e_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import userdata
import google.generativeai as genai

# Recupera a chave de forma segura dos Secrets do Colab
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

# Configura o modelo com a chave protegida
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

In [ ]:
language = 'pt'

# 1. Gravação de Áudio Com Python (e Uma Pitada de JavaScript) 🎤

In [ ]:
# Referência: https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be

from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

# Código JavaScript para gravar áudio do usuário usando a "MediaStream Recording API"
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=5):
  # Executa o código JavaScript para gravar o áudio
  display(Javascript(RECORD))
  # Recebe o áudio gravado como resultado do JavaScript
  js_result = output.eval_js('record(%s)' % (sec * 1000))
   # Decodifica o áudio em base64
  audio = b64decode(js_result.split(',')[1])
  # Salva o áudio em um arquivo
  file_name = 'request_audio.wav'
  with open(file_name, 'wb') as f:
    f.write(audio)
  # Retorna o caminho do arquivo de áudio (pasta padrão do Google Colab)
  return f'/content/{file_name}'

# Grava o áudio do usuário por um tempo determinado (padrão 5 segundos)
print('Ouvindo...\n')
record_file = record()

# Exibe o áudio gravado
display(Audio(record_file, autoplay=False))

# 2. Reconhecimento de Fala com Whisper (OpenAI) 🧠

In [ ]:
!pip install git+https://github.com/openai/whisper.git -q

In [ ]:
import whisper

# Selecione o modelo do Whisper que melhor atenda às suas necessidades:
# https://github.com/openai/whisper#available-models-and-languages
model = whisper.load_model("small")

# Transcreve o audio gravado anteriormente.
result = model.transcribe(record_file, fp16=False, language=language)
transcription = result["text"]
print(transcription)

# 3. Integração com a API do Gemini 💬

In [ ]:
# Inicializa o modelo Gemini
model_gemini = genai.GenerativeModel('gemini-2.5-flash')

# Envia a transcrição gerada pelo Whisper para o Gemini
# A variável 'transcription' foi definida na etapa 2 do seu notebook
try:
    response = model_gemini.generate_content(transcription)

    # Armazena a resposta para que a célula do gTTS possa ler
    chatgpt_response = response.text
    print(f"Gemini respondeu: {chatgpt_response}")
except Exception as e:
    print(f"Erro ao chamar a API do Gemini: {e}")

# 4. Sintetizando a Resposta do Gemini Como Voz (gTTS) 🔊

In [ ]:
!pip install gTTS

In [ ]:
from gtts import  gTTS

# Cria um objeto gTTS com a resposta gerada pelo ChatGPT e a língua que será sintetizada em voz (variável "language").
gtts_object = gTTS(text=chatgpt_response, lang=language, slow=False)

# Salva o áudio da resposta no arquivo especificado (pasta padrão do Google Colab)
response_audio = "/content/response_audio.wav"
gtts_object.save(response_audio)

# Reproduz o áudio da resposta salvo no arquivo
display(Audio(response_audio, autoplay=True))